In [1]:
import h3
import os
import polars as pl
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import folium

In [3]:
# create data directories
os.makedirs("./data/objects", exist_ok=True)
os.makedirs("./data/people", exist_ok=True)

In [4]:
# delete all files in data directories
for file in os.listdir("./data/objects"):
    os.remove(os.path.join("./data/objects", file))
for file in os.listdir("./data/people"):
    os.remove(os.path.join("./data/people", file))


In [2]:
objects = pl.read_parquet("./data/objects/")
people = pl.read_parquet("./data/population/people")
positions = pl.read_parquet("./data/population/positions")
orders = pl.read_parquet("./data//orders")
order_lines = pl.read_parquet("./data//order_lines")

orders_counts = pl.sql("SELECT status, count(*) as count FROM orders GROUP BY status").collect()
order_lines_counts = pl.sql("SELECT status, count(*) as count FROM order_lines GROUP BY status").collect()
people_counts = pl.sql("SELECT role, count(*) as count FROM people GROUP BY role").collect()
object_counts = pl.sql("SELECT label, count(*) as count FROM objects GROUP BY label").collect()

In [3]:
orders.write_delta("data/orders_delta")

In [109]:
people.write_parquet("./people.parquet")
positions.write_parquet("./positions.parquet")


In [ ]:
# Create subplot with 1 row and 2 columns
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])

# Add object counts pie chart
fig.add_trace(
    go.Pie(
        labels=object_counts['label'].to_list(),
        values=object_counts['count'].to_list(),
        name="Objects",
        title="Object Types",
        showlegend=True,
        legendgroup="objects",
    ),
    row=1, col=1
)

# Add people counts pie chart
fig.add_trace(
    go.Pie(
        labels=people_counts['role'].to_list(),
        values=people_counts['count'].to_list(),
        name="People",
        title="People Roles",
        showlegend=True,
        legendgroup="people",
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(
    title_text="Data Composition Analysis",
    height=500,
    showlegend=True,
    template="plotly_dark",
)

fig

In [42]:
trips = (
    people.select(["id", "role"])
    .filter(pl.col("role") == "courier")
    .join(positions, on="id", how="left")
    .sort(["id", "timestamp"])
    .group_by("id", maintain_order=True)
)

In [102]:
from folium.plugins import TimestampedGeoJson
import datetime

epoch = datetime.datetime.fromtimestamp(0, datetime.timezone.utc)


trips = (
    people.select(["id", "role"])
    .filter(pl.col("role") == "courier")
    .join(positions, on="id", how="left")
    .sort(["id", "timestamp"])
    .group_by("id", maintain_order=True)
)


routes = []
features = []
for id, trip in trips:
    poss = trip["position"].to_list()
    tss = trip["timestamp"].to_list()
    feature = [
        {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": poss[:idx],
            },
            "properties": {
                "times": [ts.isoformat() for ts in tss[:idx]],
                "icon": "circle",
                "iconstyle": {
                    "fillColor": "red",
                    "fillOpacity": 0.6,
                    "stroke": "false",
                    "radius": 5,
                },
            },
        }
        for idx in range(len(poss))
    ]
    features.extend(feature)
    routes.append(
        {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": poss,
            },
            "properties": {
                "times": [ts.isoformat() for ts in tss],
                "style": {
                    "color": "blue",
                    "weight": 3,
                    "opacity": 0.6,
                },
                "icon": "circle",
                "iconstyle": {
                    "fillColor": "red",
                    "fillOpacity": 0.6,
                    "stroke": "false",
                    "radius": 5,
                },
            },
        }
    )


In [69]:
poss = trip["position"].to_list()
tss = trip["timestamp"].to_list()

features = [
    {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [pos0, pos1],
        },
        "properties": {
            "times": [ts0.isoformat(), ts1.isoformat()],
            "icon": "circle",
            "iconstyle": {
                "fillColor": "red",
                "fillOpacity": 0.6,
                "stroke": "false",
                "radius": 5,
            },
        },
    }
    for (pos0, pos1), (ts0, ts1) in zip(zip(poss[:-1], poss[1:]), zip(tss[:-1], tss[1:]))
]

In [ ]:
lng, lat = -0.13381370382489707, 51.518898098201326
resolution = 6

# Create a map centered on the first person's location
m = folium.Map(
    location=[lat, lng],
    zoom_start=13,
    tiles="CartoDB dark_matter",  # Dark theme map
)

TimestampedGeoJson(
    {"type": "FeatureCollection", "features": routes},
    add_last_point=True,
    loop=True,
    auto_play=False,
    date_options="YYYY/MM/DD HH:mm:ss",
    period="PT1M",  # 5 minutes between points
    duration="PT1M",  # 1 minute display duration for each point
    loop_button=True,
).add_to(m)

m